In [1]:
pip install tqdm

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

#0. Initial Setting

In [3]:
import numpy as np
from scipy.special import logsumexp
from scipy.special import softmax
import warnings
warnings.filterwarnings("ignore")

theta1 = np.array([1.5,1,-1.9,-1.8,1]) #RS, FC1,FC2, EC

trans_true = [0.9,0.1]
obser_true = [0.2,0.3]
theta23 = trans_true + obser_true

agent = 2
za_dim =(agent+1)*(2**agent)
za_old = np.arange(za_dim,dtype = int)

TimePeriod= 150
SamplePath = 3000
SubSample = 30

z_old = np.arange(agent+1,dtype=int) #observation
a1_old = np.arange(2,dtype = int)
a2_old = np.arange(2,dtype = int)

num_discrete = 100
x_old = np.linspace(0,1,num_discrete)


In [4]:
#print(za_old//(agent+1))
print((za_old//(agent+1))//2) #a1
print((za_old//(agent+1))%2)  #a2
print(za_old%(agent+1))       #z

np.hstack([np.hstack([a1_old[i] *2 + a2_old for i in range(2)])[j] *3+z_old for j in range(4)])
print((1*2+1)*3+2)

[0 0 0 0 0 0 1 1 1 1 1 1]
[0 0 0 1 1 1 0 0 0 1 1 1]
[0 1 2 0 1 2 0 1 2 0 1 2]
11


#1. Generate Dynamic Part and Policy Part

In [5]:
def Dynamic(theta23):

  trans_val = theta23[0:2]
  obser_val = theta23[2:4]
  trans = np.zeros([2,2]) #s_t,s_{t+1}
  trans[:,0] = trans_val
  trans[:,1] = 1- trans[:,0]
  
  observe = np.zeros([2,3,3]) #s_{t+1},z_t,z_{t+1}
  for s in range(2):
    observe[s,0,1] = observe[s,1,0] = observe[s,1,2]= observe[s,2,1] = obser_val[s]
    observe[s,0,0] = 1 - observe[s,0,1] -observe[s,0,2]
    observe[s,1,1] = 1 - observe[s,1,0] -observe[s,1,2]
    observe[s,2,2] = 1 - observe[s,2,0] - observe[s,2,1]

  return trans,observe
#print(Dynamic(theta23))

def Reward(theta1,ag_i): # 3*2*2, 2,2
  rw = np.zeros([2,(agent+1)*(2**agent),2,2])#st,z*a_{t-1},a_{t,1},a_{t,2}
  for s in range(2):
    if ag_i==1:
      rw[s,:,1,0] = (za_old%(agent+1)) * theta1[s]/((2+0)**2) - theta1[2] - theta1[4] * (1-((za_old//(agent+1))//2))
      rw[s,:,1,1] = (za_old%(agent+1)) * theta1[s]/((2+1)**2) - theta1[2] - theta1[4] * (1-((za_old//(agent+1))//2))
    elif ag_i==2:
      rw[s,:,0,1] = (za_old%(agent+1)) * theta1[s]/((2+0)**2) - theta1[3] - theta1[4] * (1-((za_old//(agent+1))%2))
      rw[s,:,1,1] = (za_old%(agent+1)) * theta1[s]/((2+1)**2) - theta1[3] - theta1[4] * (1-((za_old//(agent+1))%2))    

  return rw
#print(Reward(theta1,2))

#trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}

def SigmaLambda(za_old,z_old,x_old,trans,observe,num_discrete=num_discrete,T = None): #xt,z_t,z_{t+1}
  if T==None: #generate function for Q function iteration
      x_new = np.zeros([num_discrete,3,3])
      sigma = np.zeros([num_discrete,3,3])
      x_old = np.vstack([x_old,1-x_old]).T

      for z_new in z_old:
        #for za in za_old:#range(2):
        #za = za_old%(agent+1)
        x_new_tep = x_old.dot(np.reshape(trans[:,0],[-1,1]).dot(observe[0,:,z_new].reshape([1,-1])))
        sigma[:,:,z_new] = x_new_tep + x_old.dot(np.reshape(trans[:,1],[-1,1]).dot(observe[1,:,z_new].reshape([1,-1])))
        sigma_0 = np.where(sigma[:,:,z_new]!=0)
        x_new[:,:,z_new][sigma_0] = x_new_tep[sigma_0] /sigma[:,:,z_new][sigma_0] 

      belief_f = np.floor(x_new*(num_discrete-1))/(num_discrete-1)
      belief_c = np.ceil(x_new*(num_discrete-1))/(num_discrete-1)
      iterpolate = np.zeros(belief_f.shape)
      iterpolate[np.where((belief_f-belief_c)!=0)] = (x_new-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]
  elif T==1: #generate 1 period data for data generation
      x_new = np.zeros(x_old.shape)
      sigma = np.zeros(x_old.shape)
      # #x_old = np.vstack([x_old,1-x_old]).T
      # za = za_old%(agent+1)
     
      x_new_tep = (x_old  *trans[0,0] +(1-x_old)*trans[1,0])* observe[0,za_old,z_old]
      sigma = x_new_tep + (x_old *trans[0,1] + (1-x_old)*trans[1,1] )* observe[1,za_old,z_old]
      x_new = x_new_tep/sigma

      belief_f = np.floor(x_new*(num_discrete-1))/(num_discrete-1)
      belief_c = np.ceil(x_new*(num_discrete-1))/(num_discrete-1)
      iterpolate = np.zeros(belief_f.shape)
      iterpolate[np.where((belief_f-belief_c)!=0)] = (x_new-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]
  else:      #generate whole belief for recover process
      x_new = np.zeros([T,x_old.shape[0]]) # T, N
      sigma = np.zeros([T-1,x_old.shape[0]]) # T-1 ,N
      x_new[0,:] = x_old
      #x_old = np.vstack([x_old,1-x_old]).T

      for t in range(1,T):
        # a1 = a1_old[t-1,:]
        # a2 = a2_old[t-1,:]
        za = z_old[t-1,:]
        z = z_old[t,:]
        x_new_tep = (x_new[t-1,:] *trans[0,0] + (1-x_new[t-1,:])*trans[1,0] )* observe[0,za,z]
        sigma[t-1,:] = x_new_tep + (x_new[t-1,:] *trans[0,1] + (1-x_new[t-1,:])*trans[1,1] )* observe[1,za,z]
        x_new[t,:] = x_new_tep/sigma[t-1,:]

      belief_f = np.floor(x_new*(num_discrete-1))/(num_discrete-1)
      belief_c = np.ceil(x_new*(num_discrete-1))/(num_discrete-1)
      iterpolate = np.zeros(belief_f.shape)
      iterpolate[np.where((belief_f-belief_c)!=0)] = (x_new-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      

  return sigma,x_new,[iterpolate,np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))]
#print(SigmaLambda(None,z_old,x_old,trans,observe)[0][:,0,1,0,0])

#trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
#sigma,x_new,xiterpolate = SigmaLambda(None,z_old,x_old,trans,observe)

def ValueIteration(theta1,sigma,xiterpolate, num_discrete=num_discrete,beta = 0.95,
                   gamma = 0.5772,thread_inner = 1e-5,thread_outer = 1e-5): #xt,za, a1 or xt,za, a2
  rw1,rw2 = Reward(theta1,1), Reward(theta1,2) #st,z*a_{t-1},a_{t,1},a_{t,2}
  iterpolate,belief_f,belief_c = xiterpolate[0],xiterpolate[1],xiterpolate[2]#xt,zt,z_{t+1}
  x_old = np.linspace(0,1,num=num_discrete).reshape([-1,1])
  #x_old = np.vstack([x_old,1-x_old]).T

  pi1_new = 0.5*np.ones([num_discrete,za_dim,2]) #xt,za,a1

  pi2_old = pi1_new.copy()
  #count = 0
  #pi1_old = np.zeros([2,2])
  for ol in range(1000):
    pi1_old = pi1_new.copy()
    
    Q1_new = np.zeros([num_discrete,za_dim,2]) #xt,za,a1
    Q2_new = np.zeros([num_discrete,za_dim,2]) #xt,za,a2

    for il in range(1000):
      Q2_old = Q2_new.copy() #xt,za,a2
      za = za_old%(agent+1)

      for a2 in range(2):
        #a1 = 0
        V2_a10 = []
        for z_new in z_old:
          znew_a10= (0 *2+a2)*3 + z_new
          it0,bf0,bc0 = iterpolate[:,za,z_new],belief_f[:,za,z_new],belief_c[:,za,z_new]
          #print(znew_a10,bf0)
          V2_a10.append( np.stack([it0*Q2_old[bf0,znew_a10,0] + (1-it0)*Q2_old[bc0,znew_a10,0], it0*Q2_old[bf0,znew_a10,1] + (1-it0)*Q2_old[bc0,znew_a10,1] ],axis=2))
        V2_a10 =  np.stack(V2_a10,axis=3)

        Q2_tepa10 = x_old.dot(rw2[0,:,0,a2].reshape([1,-1])) + (1-x_old).dot(rw2[1,:,0,a2].reshape([1,-1])) + beta * np.sum(sigma[:,za,:]*(gamma+logsumexp(V2_a10,axis=2)),axis=2)

        #a1 = 1
        V2_a11 = []
        for z_new in z_old:
          znew_a11= (1 *2+a2)*3 + z_new
          it1,bf1,bc1 = iterpolate[:,za,z_new],belief_f[:,za,z_new],belief_c[:,za,z_new]
          V2_a11.append( np.stack([it1*Q2_old[bf1,znew_a11,0] + (1-it1)*Q2_old[bc1,znew_a11,0], it1*Q2_old[bf1,znew_a11,1] + (1-it1)*Q2_old[bc1,znew_a11,1] ],axis=2))
        V2_a11 =  np.stack(V2_a11,axis=3)

        Q2_tepa11 = x_old.dot(rw2[0,:,1,a2].reshape([1,-1])) + (1-x_old).dot(rw2[1,:,1,a2].reshape([1,-1])) + beta * np.sum(sigma[:,za,:]*(gamma+logsumexp(V2_a11,axis=2)),axis=2)

          
        Q2_new[:,:,a2] = pi1_old[:,:,0] *Q2_tepa10 + pi1_old[:,:,1] * Q2_tepa11
        #print(it0)
      #print(il,np.max(np.abs(np.nan_to_num((Q2_new-Q2_old)/Q2_old))))
      if np.max(np.abs(np.nan_to_num((Q2_new-Q2_old)/Q2_old))) <thread_inner: 
        #print('Q2',il,np.max(np.abs(np.nan_to_num((Q2_new-Q2_old)/Q2_old))))
        break

    pi2_new = softmax(Q2_new,axis=2)   
    count = np.max(np.abs(np.nan_to_num((pi2_old-pi2_new)/pi2_old)))


    pi2_old = pi2_new.copy()
    for il in range(1000):
      Q1_old = Q1_new.copy()
      za = za_old%(agent+1)

      for a1 in range(2):
        #a2 = 0
        V1_a20 = []
        for z_new in z_old:
          znew_a20= (a1 *2+0)*3 + z_new
          it0,bf0,bc0 = iterpolate[:,za,z_new],belief_f[:,za,z_new],belief_c[:,za,z_new]
          V1_a20.append( np.stack([it0*Q1_old[bf0,znew_a20,0] + (1-it0)*Q1_old[bc0,znew_a20,0], it0*Q1_old[bf0,znew_a20,1] + (1-it0)*Q1_old[bc0,znew_a20,1] ],axis=2))
        V1_a20 =  np.stack(V1_a20,axis=3)

        Q1_tepa20 = x_old.dot(rw1[0,:,a1,0].reshape([1,-1])) + (1-x_old).dot(rw1[1,:,a1,0].reshape([1,-1])) + beta * np.sum(sigma[:,za,:]*(gamma+logsumexp(V1_a20,axis=2)),axis=2)

        #a2 = 1
        V1_a21 = []
        for z_new in z_old:
          znew_a21= (a1 *2+1)*3 + z_new
          it1,bf1,bc1 = iterpolate[:,za,z_new],belief_f[:,za,z_new],belief_c[:,za,z_new]
          V1_a21.append( np.stack([it1*Q1_old[bf1,znew_a21,0] + (1-it1)*Q1_old[bc1,znew_a21,0], it1*Q1_old[bf1,znew_a21,1] + (1-it1)*Q1_old[bc1,znew_a21,1] ],axis=2))
        V1_a21 =  np.stack(V1_a21,axis=3)

        Q1_tepa21 = x_old.dot(rw1[0,:,a1,1].reshape([1,-1])) + (1-x_old).dot(rw1[1,:,a1,1].reshape([1,-1])) + beta * np.sum(sigma[:,za,:]*(gamma+logsumexp(V1_a21,axis=2)),axis=2)

          
        Q1_new[:,:,a1] = pi2_old[:,:,0] *Q1_tepa20 + pi2_old[:,:,1] * Q1_tepa21

      #print(il,np.max(np.abs(np.nan_to_num((Q1_new-Q1_old)/Q1_old))))

      if np.max(np.abs(np.nan_to_num((Q1_new-Q1_old)/Q1_old))) <thread_inner:
        #print('Q1',il,np.max(np.abs(np.nan_to_num((Q1_new-Q1_old)/Q1_old))))
        break

    pi1_new = softmax(Q1_new,axis=2)
    
    #print('pi1 pi2',ol,np.max(np.abs(np.nan_to_num((pi1_old-pi1_new)/pi1_old))),count)

    if np.max(np.abs(np.nan_to_num((pi1_old-pi1_new)/pi1_old))) <thread_outer and count<thread_outer:
      #print(ol,np.max(np.abs((pi1_old-pi1_new)/pi1_new)))
      break
  return Q1_new,Q2_new,pi1_new,pi2_new

# trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
# sigma,x_new,xiterpolate = SigmaLambda(za_old,z_old,x_old,trans,observe)
# Q1,Q2,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)

#2. Data Generation Process

In [6]:
trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
trans-np.diag(np.ones(2))

array([[-0.1,  0.1],
       [ 0.1, -0.1]])

In [15]:
##################################Global Parameter#################################
# theta1 = np.array([3,1.9,2]) #RS, FC, EC
# theta23 = 0.2
# agent = 2
# za_dim =(agent+1)*(2**agent)
# za_old = np.arange(z_dim,dtype = int)
# TimePeriod= 150
# SamplePath = 300
# num_discrete = 100
# z_old = np.arange(agent+1,dtype=int) #observation
# a1_old = np.arange(2,dtype = int)
# a2_old = np.arange(2,dtype = int)
###################################################################################
from tqdm import tqdm 

def DataGeneration(theta1 = theta1,theta23=theta23,T=TimePeriod,N=SamplePath,N_sub = SubSample):
  # Assume start from good condition and highest market size(2), agents entry market(1) at time 0

  trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
  sigma,x_new,xiterpolate = SigmaLambda(None,z_old,x_old,trans,observe)
  _,_,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)#xt,za, a1 or xt,za, a2


  condition = np.zeros([T,N],dtype= int)
  condition[0,:] = np.random.choice(np.array(2,dtype=int),size=N,p=[1/2,1/2])

  market_size = np.zeros([T,N],dtype = int) #observation state
  market_size[0,:] = np.random.choice(np.array(3,dtype=int),size=N,p=[1/3,1/3,1/3])

  belief = np.zeros([T,N])
  belief[0,:] = np.kron(np.linspace(0,1,num=int(N/N_sub)),np.ones(N_sub))
  sigma_val = np.ones([T-1,N])
  # belief_f = np.floor(belief[0,:]*(num_discrete-1))/(num_discrete-1)
  # belief_c = np.ceil(belief[0,:]*(num_discrete-1))/(num_discrete-1)
  # iterpolate = np.zeros(belief_f.shape)
  # iterpolate[np.where((belief_f-belief_c)!=0)] = (belief[0,:]-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
  # belief_f,belief_c = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

  action1 = np.ones([T,N],dtype = int)   #agent1's action
  action2 = np.ones([T,N],dtype = int)   #agent2's action
  # za = (np.zeros(N,dtype=int) *2 + np.zeros(N,dtype=int))*3+market_size[0,:]
  # action_mat1 = np.random.random(size=N)
  # p1 = iterpolate * pi1[belief_f,za,0] + (1-iterpolate) * pi1[belief_c,za,0]
  # action1[0,:] = np.int_(p1<=action_mat1)
  
  # action_mat2 = np.random.random(size=N)
  # p2 = iterpolate * pi2[belief_f,za,0] + (1-iterpolate) * pi2[belief_c,za,0]
  # action2[0,:] = np.int_(p2<=action_mat2)


  for t in tqdm(range(1,T)):
    #generate new condition #s_{t+1}
    cd_mat = np.random.random(size=N)
    pc = trans[condition[t-1,:],0]
    condition[t,:] = np.int_(pc<=cd_mat)
   
    # generate new market size #z_{t+1}
    ms_mat = np.random.random(size=N)
    pt = observe[condition[t,:],market_size[t-1,:],:]
    market_size[t,:] = np.int_(pt[:,0]<=ms_mat) +np.int_(pt[:,0]+pt[:,1]<=ms_mat)

    # update new belief #x_t
    za = market_size[t-1,:]
    z = market_size[t,:]
    x_new_tep = (belief[t-1,:] *trans[0,0] + (1-belief[t-1,:])*trans[1,0] )* observe[0,za,z]
    #print(x_new_tep)
    sigma_val[t-1,:] = x_new_tep + (belief[t-1,:] *trans[0,1] + (1-belief[t-1,:])*trans[1,1] )* observe[1,za,z]
    belief[t,:] = x_new_tep/sigma_val[t-1,:]

    belief_f = np.floor(belief[t,:]*(num_discrete-1))/(num_discrete-1)
    belief_c = np.ceil(belief[t,:]*(num_discrete-1))/(num_discrete-1)
    iterpolate = np.zeros(belief_f.shape)
    iterpolate[np.where((belief_f-belief_c)!=0)] = (belief[t,:]-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
    belief_f,belief_c = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

    # generate new action for both agents #a_{t+1}
    za = (action1[t-1,:] *2 + action2[t-1,:])*3+market_size[t,:]
    action_mat1 = np.random.random(size=N)
    p1 = iterpolate * pi1[belief_f,za,0] + (1-iterpolate) * pi1[belief_c,za,0]
    action1[t,:] = np.int_(p1<=action_mat1)
    
    action_mat2 = np.random.random(size=N)
    p2 = iterpolate * pi2[belief_f,za,0] + (1-iterpolate) * pi2[belief_c,za,0]
    action2[t,:] = np.int_(p2<=action_mat2)

  return market_size, action1,action2 ,belief,sigma_val,condition


market_size, action1,action2 ,belief,sigma_val,condition = DataGeneration()
#haha = SigmaLambda()

100%|██████████| 149/149 [00:00<00:00, 1587.88it/s]


In [16]:
np.sum(condition)/np.prod(condition.shape)

0.4993533333333333

#3.1 Recover Dynamic Part

In [17]:
theta23

[0.9, 0.1, 0.2, 0.3]

In [18]:
from scipy.optimize import minimize
ini_x0 = 0.85*np.array(theta23)
print('start from:',ini_x0 )

def func1(theta23,market_size=market_size, action1=action1,action2 = action2,T=TimePeriod ,x0 = belief[0,:]):
  trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
  sigma_val,_,_ = SigmaLambda(None,market_size,x0,trans,observe,T=T)

  # sigma_val,_,_ = SigmaLambda(None,market_size,x_old,action1,action2,theta23= theta23,num_discrete=num_discrete,T = T)
  #print(observe[market_size[0:-1,:],market_size[1:T,:]])
  res = -np.sum(np.log(sigma_val))
  print(theta23,res)
  return res

res1 = minimize(func1,ini_x0, bounds=[[0.00001,0.99999],[0.00001,0.99999],[0.00001,0.49999],[0.00001,0.49999]])
#trans_est = np.vstack([res1.x,1-res1.x]).T.reshape([2,2,2,2])
print(res1)

start from: [0.765 0.085 0.17  0.255]
[0.765 0.085 0.17  0.255] 323136.5406994275
[0.76500001 0.085      0.17       0.255     ] 323136.5407416256
[0.765      0.08500001 0.17       0.255     ] 323136.54082697723
[0.765      0.085      0.17000001 0.255     ] 323136.5405416366
[0.765      0.085      0.17       0.25500001] 323136.54016561183
[1.0000e-05 1.0000e-05 4.9999e-01 4.9999e-01] 1064806.0344498577
[1.0010e-05 1.0000e-05 4.9999e-01 4.9999e-01] 1064806.0344498577
[1.0000e-05 1.0010e-05 4.9999e-01 4.9999e-01] 1064806.0344498577
[1.0000000e-05 1.0000000e-05 4.9999001e-01 4.9999000e-01] 1064806.035195402
[1.0000000e-05 1.0000000e-05 4.9999000e-01 4.9999001e-01] 1064880.6254889746
[0.50445309 0.05605337 0.28239085 0.33844081] 335944.14442502375
[0.5044531  0.05605337 0.28239085 0.33844081] 335944.14438422705
[0.50445309 0.05605338 0.28239085 0.33844081] 335944.1441176102
[0.50445309 0.05605337 0.28239086 0.33844081] 335944.1448034494
[0.50445309 0.05605337 0.28239085 0.33844082] 335944.1

#3.2 Recover Reward Part

In [19]:
theta1.reshape(-1)

array([ 1.5,  1. , -1.9, -1.8,  1. ])

In [20]:
ini_x0 = 0.85*theta1.reshape(-1)
print('start from:',ini_x0 )

trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
_,belief_val,_ = SigmaLambda(None,market_size,belief[0,:],trans,observe,T=TimePeriod)
sigma,x_new,xiterpolate = SigmaLambda(None,z_old,x_old,trans,observe)

def func2(theta1,sigma = sigma,xiterpolate=xiterpolate,belief_val=belief_val,market_size=market_size,action1 = action1,action2 = action2,T=TimePeriod,x0 = belief[0,:]):
  #theta1 = theta1.reshape([2,2,2])  
 
  _,_,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)
  
  za = (action1[0:-1,:]*2 + action2[0:-1,:])*3 + market_size[1:T,:]


  belief_f = np.floor(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
  belief_c = np.ceil(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
  iterpolate = np.zeros(belief_f.shape)
  iterpolate[np.where((belief_f-belief_c)!=0)] = (belief_val[1:T,:]-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
  belief_f,belief_c = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

  p1 = iterpolate*pi1[belief_f,za,action1[1:T,:]] +(1- iterpolate)*pi1[belief_c,za,action1[1:T,:]]
  p2 = iterpolate*pi2[belief_f,za,action2[1:T,:]]+(1- iterpolate)*pi2[belief_c,za,action2[1:T,:]]
  res = - np.sum(np.log(p1)) -np.sum(np.log(p2))
  print(theta1,res)
  return res
# ineq_cons = {'type': 'ineq',
#              'fun' : lambda x: np.array([x[0] - x[1]])}
res2 = minimize(func2,ini_x0,bounds = [(1e-6,100),(1e-6,100),(-100,-1e-6),(-100,-1e-6),(1e-6,100)])#,constraints=ineq_cons)#,(1e-6,100)])#method='Nelder-Mead')
print(res2)

start from: [ 1.275  0.85  -1.615 -1.53   0.85 ]
[ 1.275  0.85  -1.615 -1.53   0.85 ] 203200.75029070658
[ 1.27500001  0.85       -1.615      -1.53        0.85      ] 203200.75027452485
[ 1.275       0.85000001 -1.615      -1.53        0.85      ] 203200.75027434994
[ 1.275       0.85       -1.61499999 -1.53        0.85      ] 203200.7504297292
[ 1.275       0.85       -1.615      -1.52999999  0.85      ] 203200.7504346662
[ 1.275       0.85       -1.615      -1.53        0.85000001] 203200.75008956512
[ 100.  100. -100. -100.  100.] 10105531.111111118
[ 100.00000001  100.         -100.         -100.          100.        ] 10105531.111137278
[ 100.          100.00000001 -100.         -100.          100.        ] 10105531.111137493
[ 100.          100.          -99.99999999 -100.          100.        ] 10105531.110863615
[ 100.          100.         -100.          -99.99999999  100.        ] 10105531.110830653
[ 100.          100.         -100.         -100.          100.00000001] 10105

In [21]:
ini_x0 = 0.85*theta1.reshape(-1)
print('start from:',ini_x0 )

trans,observe = Dynamic(res1.x)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
_,belief_val,_ = SigmaLambda(None,market_size,belief[0,:],trans,observe,T=TimePeriod)
sigma,x_new,xiterpolate = SigmaLambda(None,z_old,x_old,trans,observe)

def func2(theta1,sigma = sigma,xiterpolate=xiterpolate,belief_val=belief_val,market_size=market_size,action1 = action1,action2 = action2,T=TimePeriod,x0 = belief[0,:]):
  #theta1 = theta1.reshape([2,2,2])  
 
  _,_,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)
  
  za = (action1[0:-1,:]*2 + action2[0:-1,:])*3 + market_size[1:T,:]


  belief_f = np.floor(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
  belief_c = np.ceil(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
  iterpolate = np.zeros(belief_f.shape)
  iterpolate[np.where((belief_f-belief_c)!=0)] = (belief_val[1:T,:]-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
  belief_f,belief_c = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

  p1 = iterpolate*pi1[belief_f,za,action1[1:T,:]] +(1- iterpolate)*pi1[belief_c,za,action1[1:T,:]]
  p2 = iterpolate*pi2[belief_f,za,action2[1:T,:]]+(1- iterpolate)*pi2[belief_c,za,action2[1:T,:]]
  res = - np.sum(np.log(p1)) -np.sum(np.log(p2))
  print(theta1,res)
  return res
# ineq_cons = {'type': 'ineq',
#              'fun' : lambda x: np.array([x[0] - x[1]])}
res2 = minimize(func2,ini_x0,bounds = [(1e-6,100),(1e-6,100),(-100,-1e-6),(-100,-1e-6),(1e-6,100)])#,constraints=ineq_cons)#,(1e-6,100)])#method='Nelder-Mead')
print(res2)

start from: [ 1.275  0.85  -1.615 -1.53   0.85 ]
[ 1.275  0.85  -1.615 -1.53   0.85 ] 203180.87287797368
[ 1.27500001  0.85       -1.615      -1.53        0.85      ] 203180.8728613811
[ 1.275       0.85000001 -1.615      -1.53        0.85      ] 203180.87286214565
[ 1.275       0.85       -1.61499999 -1.53        0.85      ] 203180.87301671278
[ 1.275       0.85       -1.615      -1.52999999  0.85      ] 203180.8730216229
[ 1.275       0.85       -1.615      -1.53        0.85000001] 203180.87267721977
[ 100.  100. -100. -100.  100.] 10105531.111111116
[ 100.00000001  100.         -100.         -100.          100.        ] 10105531.111138027
[ 100.          100.00000001 -100.         -100.          100.        ] 10105531.111136744
[ 100.          100.          -99.99999999 -100.          100.        ] 10105531.110863615
[ 100.          100.         -100.          -99.99999999  100.        ] 10105531.110830653
[ 100.          100.         -100.         -100.          100.00000001] 10105

In [ ]:
# ini_x0 = theta1.reshape(-1)
# print('start from:',ini_x0 )

# trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
# _,belief_val,_ = SigmaLambda(None,market_size,belief[0,:],trans,observe,T=TimePeriod)
# sigma,x_new,xiterpolate = SigmaLambda(None,z_old,x_old,trans,observe)

# def func2(theta1,sigma = sigma,xiterpolate=xiterpolate,belief_val=belief_val,market_size=market_size,action1 = action1,action2 = action2,T=TimePeriod,x0 = belief[0,:]):
#   #theta1 = theta1.reshape([2,2,2])  
 
#   _,_,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)
  
#   za = (action1[0:-1,:]*2 + action2[0:-1,:])*3 + market_size[1:T,:]


#   belief_f = np.floor(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
#   belief_c = np.ceil(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
#   iterpolate = np.zeros(belief_f.shape)
#   iterpolate[np.where((belief_f-belief_c)!=0)] = (belief_val[1:T,:]-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
#   belief_f,belief_c = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

#   p1 = iterpolate*pi1[belief_f,za,action1[1:T,:]] +(1- iterpolate)*pi1[belief_c,za,action1[1:T,:]]
#   p2 = iterpolate*pi2[belief_f,za,action2[1:T,:]]+(1- iterpolate)*pi2[belief_c,za,action2[1:T,:]]
#   res = - np.sum(np.log(p1)) -np.sum(np.log(p2))
#   print(theta1,res)
#   return res
# # ineq_cons = {'type': 'ineq',
# #              'fun' : lambda x: np.array([x[0] - x[1]])}
# res2 = minimize(func2,ini_x0,bounds = [(1e-6,100),(1e-6,100),(-100,-1e-6),(-100,-1e-6),(1e-6,100)])#,constraints=ineq_cons)#,(1e-6,100)])#method='Nelder-Mead')
# print(res2)

start from: [ 2.   1.  -1.9 -1.8  2. ]
[ 2.   1.  -1.9 -1.8  2. ] 45153.64099148799
[ 2.00000001  1.         -1.9        -1.8         2.        ] 45153.6409915609
[ 2.          1.00000001 -1.9        -1.8         2.        ] 45153.640991539345
[ 2.          1.         -1.89999999 -1.8         2.        ] 45153.64099035924
[ 2.          1.         -1.9        -1.79999999  2.        ] 45153.640990996086
[ 2.          1.         -1.9        -1.8         2.00000001] 45153.64099255887
[ 1.e-06  1.e-06 -1.e-06 -1.e-06  1.e-06] 275872.2678200912
[ 1.01e-06  1.00e-06 -1.00e-06 -1.00e-06  1.00e-06] 275872.267585629
[ 1.00e-06  1.01e-06 -1.00e-06 -1.00e-06  1.00e-06] 275872.2677060706
[ 1.0e-06  1.0e-06 -9.9e-07 -1.0e-06  1.0e-06] 275872.2687676414
[ 1.0e-06  1.0e-06 -1.0e-06 -9.9e-07  1.0e-06] 275872.2687621919
[ 1.00e-06  1.00e-06 -1.00e-06 -1.00e-06  1.01e-06] 275872.266957799
[ 1.99646951  0.99823475 -1.89664603 -1.79682256  1.99646951] 45152.9182252409
[ 1.99646952  0.99823475 -1.89664603 -

In [ ]:
# ini_x0 = 0.85*theta1.reshape(-1)
# print('start from:',ini_x0 )

# trans,observe = Dynamic(theta23)  #s_t,s_{t+1} #s_{t+1},z_t,z_{t+1}
# _,belief_val,_ = SigmaLambda(None,market_size,belief[0,:],trans,observe,T=TimePeriod)
# sigma,x_new,xiterpolate = SigmaLambda(None,z_old,x_old,trans,observe)

# def func2(theta1,sigma = sigma,xiterpolate=xiterpolate,belief_val=belief_val,market_size=market_size,action1 = action1,action2 = action2,T=TimePeriod,x0 = belief[0,:]):
#   #theta1 = theta1.reshape([2,2,2])  
 
#   _,_,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)
  
#   za = (action1[0:-1,:]*2 + action2[0:-1,:])*3 + market_size[1:T,:]


#   belief_f = np.floor(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
#   belief_c = np.ceil(belief_val[1:T,:]*(num_discrete-1))/(num_discrete-1)
#   iterpolate = np.zeros(belief_f.shape)
#   iterpolate[np.where((belief_f-belief_c)!=0)] = (belief_val[1:T,:]-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      
#   belief_f,belief_c = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

#   p1 = iterpolate*pi1[belief_f,za,action1[1:T,:]] +(1- iterpolate)*pi1[belief_c,za,action1[1:T,:]]
#   p2 = iterpolate*pi2[belief_f,za,action2[1:T,:]]+(1- iterpolate)*pi2[belief_c,za,action2[1:T,:]]
#   res = - np.sum(np.log(p1)) -np.sum(np.log(p2))
#   print(theta1,res)
#   return res
# # ineq_cons = {'type': 'ineq',
# #              'fun' : lambda x: np.array([x[0] - x[1]])}
# res2 = minimize(func2,ini_x0,bounds = [(1e-6,100),(1e-6,100),(-100,-1e-6),(-100,-1e-6),(1e-6,100)])#,constraints=ineq_cons)#,(1e-6,100)])#method='Nelder-Mead')
# print(res2)

start from: [ 1.7    0.85  -1.615 -1.53   1.7  ]
[ 1.7    0.85  -1.615 -1.53   1.7  ] 47060.45420142137
[ 1.70000001  0.85       -1.615      -1.53        1.7       ] 47060.45419555539
[ 1.7         0.85000001 -1.615      -1.53        1.7       ] 47060.454198550055
[ 1.7         0.85       -1.61499999 -1.53        1.7       ] 47060.45424057009
[ 1.7         0.85       -1.615      -1.52999999  1.7       ] 47060.45424502103
[ 1.7         0.85       -1.615      -1.53        1.70000001] 47060.45414143735
[ 100.  100. -100. -100.  100.] 1906369.444444444
[ 100.00000001  100.         -100.         -100.          100.        ] 1906369.4444508827
[ 100.          100.00000001 -100.         -100.          100.        ] 1906369.4444476296
[ 100.          100.          -99.99999999 -100.          100.        ] 1906369.4443969945
[ 100.          100.         -100.          -99.99999999  100.        ] 1906369.444391545
[ 100.          100.         -100.         -100.          100.00000001] 1906369.44